In [55]:
# -*- coding: utf-8 -*-
# <nbformat>3.0</nbformat>

# <codecell>
from __future__ import print_function, absolute_import, division
import numpy as np
from scipy import optimize, interpolate, integrate
from scipy import constants as const
import pandas as pd

# <markdowncell>

# ##AbelNoble: 
# 
# - thermodynamic class that Abel-Nobel EOS for various necessary state variables
# - default $b$, $\gamma$, and $M$ set for H<sub>2</sub>, and default ideal gas constant (don't know why anyone would ever want to change this--maybe to check different precisions??)
# - ideal gas law by setting $b = 0$
# - I can envision writing other classes with the same functions (i.e. something that calls refprop to find the same variables)
# 
class EOS:
    def __init__(self, species = ['hydrogen'], x = [1.]):
        '''
        Class that uses CoolProp for equation of state calculations.
        '''
        from CoolProp import CoolProp
        self.cp = CoolProp
        if len(species) == 1:
            self.spec = species[0]
        else:
            if size(x) != size(species):
                warnings.warn('mole fractions and species lists not the same length')
            s = 'REFPROP-MIX:'
            for g, xspec in zip(species, x):
                s += g + '[' + str(xspec) + ']' + '&'
            self.spec = s[:-1]

    def h(self, T, P):
        '''enthalpy (J/kg) of a gas at temperature T (K) and pressure P (Pa)'''
        return self.cp.PropsSI('H', 'T', T, 'P', P, self.spec)
            
    def errS(self, T0, rho0, T1, rho1):
        '''
        returns the error in entropy (J/kg-K) between 2 states specified by the 
        temperatures and densities.
        '''
        s0 = self.cp.PropsSI('S', 'T', T0, 'D', rho0, self.spec)
        s1 = self.cp.PropsSI('S', 'T', T1, 'D', rho1, self.spec)
        return s1 / s0 - 1.
    
    def rho_Iflow(self, rho0, Ma = 1, Ma0 = 0):
        '''
        solves for the density of gas after isentropic flow
        
        Parameters
        ----------
        rho0 - density before exansion (kg/m^3)
        Ma - mach number during epansion 
        
        Returns
        -------
        rho - density after expansion (kg/m^3)
        '''
        from scipy import optimize
        T0 = 273.
        sf = self.cp.State(self.spec, {'T':T0, 'D':rho0})
        def errT(T0):
            s0 = self.cp.State(self.spec, {'T':T0, 'D':rho0})
            def errh(h):
                sf.update({'S':s0.get_s(), 'H':h})
                hf = s0.get_h()*1e3 + (Ma0*s0.get_speed_sound())**2/2 - (Ma*sf.get_speed_sound())**2/2.
                return hf/1000. - h
            hf = optimize.root(errh, s0.get_h())
            Tf = self.T_IflowV(T0, 0, s0.p*1e3, Ma*sf.get_speed_sound(), sf.p*1e3)
            return Tf - sf.T
        T0 = optimize.root(errT, T0)
        return sf.get_rho()
       
    def T_Iflow(self, T0, rho, Ma = 1, Ma0 = 0):
        '''
        solves for the temperature of gas after an isentropic flow
        
        Parameters
        ----------
        T0 - temperature before expansion (K)
        rho - density after expansion (kg/m^3)
        Ma - mach number during expansion
        Ma0 - mach number before expansion
        
        Returns
        -------
        T - temperature after expansion(K)
        '''
        from scipy import optimize
        def errS(x):
            [P0, Pf] = x
            P0, Pf = float(P0), float(Pf)
            s0 = self.cp.State(self.spec, {'P':P0/1e3, 'T':T0})
            sf = self.cp.State(self.spec, {'P':Pf/1e3, 'D':rho})
            hf = s0.get_h()*1e3 + (Ma0*s0.get_speed_sound())**2/2. - (Ma*sf.get_speed_sound())**2/2.
            return array([sf.get_s() - s0.get_s(), sf.get_h()*1e3 - hf])
        P0, Pf = optimize.fsolve(errS, array([101325., 101325.]))
        return self.cp.PropsSI('T', 'P', Pf, 'D', rho, self.spec)
    
    def P_Iflow(self, P0, rho, Ma = 1, Ma0 = 0):
        '''
        solves for the temperature of gas after an isentropic flow
        
        Parameters
        ----------
        P0 - Pressure before expansion (Pa)
        rho - density after expansion (kg/m^3)
        Ma - mach number during expansion
        Ma0 - mach number before expansion
        
        Returns
        -------
        P - pressure after expansion(Pa)
        '''
        from scipy import optimize
        def errS(x):
            [T0, Pf] = x
            T0, Pf = float(T0), float(Pf)
            s0 = self.cp.State(self.spec, {'P':P0/1e3, 'T':T0})
            sf = self.cp.State(self.spec, {'P':Pf/1e3, 'D':rho})
            hf = s0.get_h()*1e3 + (Ma0*s0.get_speed_sound())**2/2. - (Ma*sf.get_speed_sound())**2/2.
            return array([sf.get_s() - s0.get_s(), sf.get_h()*1e3 - hf])
        T0, Pf = optimize.fsolve(errS, array([273., 101325.]))
        return Pf
    
    def T_IflowV(self, T0, V0, P0, V, P):
        '''
        solves for the temperature of gas after an isentropic flow
        
        Parameters
        ----------
        T0 - temperature before expansion (K)
        V0 - velocity before expansion (m/s)
        P0 - pressure before expansion (Pa)
        V - velocity after expansion (m/s)
        P - pressure after expansion (Pa)
        
        Returns
        -------
        T - temperature after expansion(K)
        '''
        h = self.cp.PropsSI('H', 'T', T0, 'P', P0, self.spec) + V0**2/2. - V**2/2.
        return self.cp.PropsSI('T', 'P', P, 'H', h, self.spec)
    
    def T_IE(self, T0, rho0, rho):
        from scipy import optimize
        '''
        solves for the temperature of gas after an isentropic expansion, given an old and new 
        density
        
        Parameters
        ----------
        T0 - temperature before expansion (K)
        rho0 - density before expansion (kg/m^3)
        rho - density after expansion (kg/m^3)
        
        Returns
        -------
        T - temperature after expansion(K)
        '''
        S0 = self.cp.PropsSI('S', 'T', T0, 'D', rho0, self.spec)
        def errD(T):
            return rho - self.cp.PropsSI('D', 'T', T, 'S', S0, self.spec)
        return optimize.newton(errD, T0)
    
    def P(self, T, rho):
        '''
        returns the pressure given the temperature and density
        
        Parameters
        ----------
        T - temperature (K)
        rho - density (kg/m^3)
        
        Returns
        -------
        P - pressure (Pa)
        '''
        return self.cp.PropsSI('P', 'T', T, 'D', rho, self.spec)
    
    def T(self, P, rho):
        '''
        returns the temperature given the temperature and pressure
        
        Parameters
        ----------
        P - pressure (Pa)
        rho - density (kg/m^3)
        
        Returns
        -------
        T - temperature (K)
        '''
        return self.cp.PropsSI('T', 'D', rho, 'P', P, self.spec)
    
    def rho(self, T, P):
        '''
        returns the pressure given the temperature and pressure
        
        Parameters
        ----------
        T - temperature (K)
        P - pressure (Pa)
        
        Returns
        -------
        rho - density (kg/m^3)
        '''
        return self.cp.PropsSI('D', 'T', T, 'P', P, self.spec)
 
    def a(self, T, rho):
        '''
        returns the speed of sound given the temperature and density
        
        Parameters
        ----------
        T - temperature (K)
        P - pressure (Pa)
        
        Returns
        -------
        a - speed of sound (m/s)
        '''
        return self.cp.PropsSI('A', 'T', T, 'D', rho, self.spec)
    
    def T_IflowSonic(self, T0, P0, P):
        from scipy import optimize
        '''
        solves for the temperature of gas after an isentropic flow, assuming that the entrance
        and exit velocities are sonic.
        
        Parameters
        ----------
        T0 - temperature before expansion (K)
        P0 - pressure before expansion (Pa)
        P - pressure after expansion (Pa)
        
        Returns
        -------
        T - temperature after expansion(K)
        '''
        V0 = self.a(T0, self.rho(T0, P0))
        def errH(T):
            V = self.a(T, self.rho(T, P))
            h = self.cp.PropsSI('H', 'T', T0, 'P', P0, self.spec) + V0**2/2. - V**2/2.
            return h - self.cp.PropsSI('H', 'T', T, 'P', P, self.spec)
        T = optimize.newton(errH, T0)
        return T
    
    def T_AdFlow(self, T0, P0, Ma0, P, Ma):
        from scipy import optimize
        '''
        solves for the temperature of gas after an isentropic flow, assuming that the entrance
        and exit velocities are sonic.
        
        Parameters
        ----------
        T0 - temperature before expansion (K)
        P0 - pressure before expansion (Pa)
        P - pressure after expansion (Pa)
        
        Returns
        -------
        T - temperature after expansion(K)
        '''
        V0 = Ma0*self.a(T0, self.rho(T0, P0))
        def errH(T):
            V = Ma*self.a(T, self.rho(T, P))
            h = self.cp.PropsSI('H', 'T', T0, 'P', P0, self.spec) + V0**2/2. - V**2/2.
            return h - self.cp.PropsSI('H', 'T', T, 'P', P, self.spec)
        T = optimize.newton(errH, T0)
        return T

        
    
    
   
    
    
    
    
        
    
        
    
        
    
    
 

    
    # def _err_adiabatic_out(self, T1, P1, T2, P2, dm_per_m):
        # '''
        # returns the difference in internal energy (J/kg) between 2 states specified by the 
        # temperatures and pressures.
        
        # Parameters
        # ----------
        # T1: float
            # temperature of gas at point 1 (K)
        # P1: float
            # pressure of gas at point 1 (Pa)
        # T2: float
            # temperature of gas at point 2 (K)
        # P2: float
            # Pressure of gas at point 2 (Pa)
        # dm_per_m: float
            # relative mass difference between state 2 and 1 (m2 - m1)/m1
        # Returns
        # -------
        # err_adiabatic: float
            # error in energy balance between the two different states
        # '''
        # U1, H1, rho1 = self._cp.PropsSI(['U', 'H', 'D'], 'T', T1, 'P', P1, self.spec)
        # U2, rho2 = self._cp.PropsSI(['U', 'D'], 'T', T2, 'P', P2, self.spec)
        # return (rho2*U2 - rho1*U1) / (dm_per_m*rho1*H1)-1
            
   
        
    
            

# <codecell>





In [61]:
from __future__ import print_function, absolute_import, division
import numpy as np
from scipy import optimize, interpolate, integrate
from scipy import constants as const
import pandas as pd

# <markdowncell>

# ##AbelNoble: 
# 
# - thermodynamic class that Abel-Nobel EOS for various necessary state variables
# - default $b$, $\gamma$, and $M$ set for H<sub>2</sub>, and default ideal gas constant (don't know why anyone would ever want to change this--maybe to check different precisions??)
# - ideal gas law by setting $b = 0$
# - I can envision writing other classes with the same functions (i.e. something that calls refprop to find the same variables)
# 
class EOS2:
    def __init__(self, species = ['hydrogen'], x = [1.]):
        '''
        Class that uses CoolProp for equation of state calculations.
        '''
        from CoolProp import CoolProp
        self.cp = CoolProp
        if len(species) == 1:
            self.spec = species[0]
        else:
            if size(x) != size(species):
                warnings.warn('mole fractions and species lists not the same length')
            s = 'REFPROP-MIX:'
            for g, xspec in zip(species, x):
                s += g + '[' + str(xspec) + ']' + '&'
            self.spec = s[:-1]

    def h(self, T, P):
        '''enthalpy (J/kg) of a gas at temperature T (K) and pressure P (Pa)'''
        return self.cp.PropsSI('H', 'T', T, 'P', P, self.spec)
            
    def errS(self, T0, rho0, T1, rho1):
        '''
        returns the error in entropy (J/kg-K) between 2 states specified by the 
        temperatures and densities.
        '''
        s0 = self.cp.PropsSI('S', 'T', T0, 'D', rho0, self.spec)
        s1 = self.cp.PropsSI('S', 'T', T1, 'D', rho1, self.spec)
        return s1 / s0 - 1.
    
    def rho_Iflow(self, rho0, Ma = 1, Ma0 = 0):
        '''
        solves for the density of gas after isentropic flow
        
        Parameters
        ----------
        rho0 - density before exansion (kg/m^3)
        Ma - mach number during epansion 
        
        Returns
        -------
        rho - density after expansion (kg/m^3)
        '''
        from scipy import optimize
        T0 = 273.
        sf = self.cp.State(self.spec, {'T':T0, 'D':rho0})
        def errT(T0):
            s0 = self.cp.State(self.spec, {'T':T0, 'D':rho0})
            def errh(h):
                sf.update({'S':s0.get_s(), 'H':h})
                hf = s0.get_h()*1e3 + (Ma0*s0.get_speed_sound())**2/2 - (Ma*sf.get_speed_sound())**2/2.
                return hf/1000. - h
            hf = optimize.root(errh, s0.get_h())
            Tf = self.T_IflowV(T0, 0, s0.p*1e3, Ma*sf.get_speed_sound(), sf.p*1e3)
            return Tf - sf.T
        T0 = optimize.root(errT, T0)
        return sf.get_rho()
       
    def T_Iflow(self, T0, rho, Ma = 1, Ma0 = 0):
        '''
        solves for the temperature of gas after an isentropic flow
        
        Parameters
        ----------
        T0 - temperature before expansion (K)
        rho - density after expansion (kg/m^3)
        Ma - mach number during expansion
        Ma0 - mach number before expansion
        
        Returns
        -------
        T - temperature after expansion(K)
        '''
        from scipy import optimize
        def errS(x):
            [P0, Pf] = x
            P0, Pf = float(P0), float(Pf)
            s0 = self.cp.State(self.spec, {'P':P0/1e3, 'T':T0})
            sf = self.cp.State(self.spec, {'P':Pf/1e3, 'D':rho})
            hf = s0.get_h()*1e3 + (Ma0*s0.get_speed_sound())**2/2. - (Ma*sf.get_speed_sound())**2/2.
            return array([sf.get_s() - s0.get_s(), sf.get_h()*1e3 - hf])
        P0, Pf = optimize.fsolve(errS, array([101325., 101325.]))
        return self.cp.PropsSI('T', 'P', Pf, 'D', rho, self.spec)
    
    def P_Iflow(self, P0, rho, Ma = 1, Ma0 = 0):
        '''
        solves for the temperature of gas after an isentropic flow
        
        Parameters
        ----------
        P0 - Pressure before expansion (Pa)
        rho - density after expansion (kg/m^3)
        Ma - mach number during expansion
        Ma0 - mach number before expansion
        
        Returns
        -------
        P - pressure after expansion(Pa)
        '''
        from scipy import optimize
        def errS(x):
            [T0, Pf] = x
            T0, Pf = float(T0), float(Pf)
            s0 = self.cp.State(self.spec, {'P':P0/1e3, 'T':T0})
            sf = self.cp.State(self.spec, {'P':Pf/1e3, 'D':rho})
            hf = s0.get_h()*1e3 + (Ma0*s0.get_speed_sound())**2/2. - (Ma*sf.get_speed_sound())**2/2.
            return array([sf.get_s() - s0.get_s(), sf.get_h()*1e3 - hf])
        T0, Pf = optimize.fsolve(errS, array([273., 101325.]))
        return Pf
    
    def T_IflowV(self, T0, V0, P0, V, P):
        '''
        solves for the temperature of gas after an isentropic flow
        
        Parameters
        ----------
        T0 - temperature before expansion (K)
        V0 - velocity before expansion (m/s)
        P0 - pressure before expansion (Pa)
        V - velocity after expansion (m/s)
        P - pressure after expansion (Pa)
        
        Returns
        -------
        T - temperature after expansion(K)
        '''
        h = self.cp.PropsSI('H', 'T', T0, 'P', P0, self.spec) + V0**2/2. - V**2/2.
        return self.cp.PropsSI('T', 'P', P, 'H', h, self.spec)
    
    def T_IE(self, T0, rho0, rho):
        from scipy import optimize
        '''
        solves for the temperature of gas after an isentropic expansion, given an old and new 
        density
        
        Parameters
        ----------
        T0 - temperature before expansion (K)
        rho0 - density before expansion (kg/m^3)
        rho - density after expansion (kg/m^3)
        
        Returns
        -------
        T - temperature after expansion(K)
        '''
        S0 = self.cp.PropsSI('S', 'T', T0, 'D', rho0, self.spec)
        def errD(T):
            return rho - self.cp.PropsSI('D', 'T', T, 'S', S0, self.spec)
        return optimize.newton(errD, T0)
    
    def P(self, T, rho):
        '''
        returns the pressure given the temperature and density
        
        Parameters
        ----------
        T - temperature (K)
        rho - density (kg/m^3)
        
        Returns
        -------
        P - pressure (Pa)
        '''
        return self.cp.PropsSI('P', 'T', T, 'D', rho, self.spec)
    
    def T(self, P, rho):
        '''
        returns the temperature given the temperature and pressure
        
        Parameters
        ----------
        P - pressure (Pa)
        rho - density (kg/m^3)
        
        Returns
        -------
        T - temperature (K)
        '''
        return self.cp.PropsSI('T', 'D', rho, 'P', P, self.spec)
    
    def rho(self, T, P):
        '''
        returns the pressure given the temperature and pressure
        
        Parameters
        ----------
        T - temperature (K)
        P - pressure (Pa)
        
        Returns
        -------
        rho - density (kg/m^3)
        '''
        return self.cp.PropsSI('D', 'T', T, 'P', P, self.spec)
 
    def a(self, T, rho):
        '''
        returns the speed of sound given the temperature and density
        
        Parameters
        ----------
        T - temperature (K)
        P - pressure (Pa)
        
        Returns
        -------
        a - speed of sound (m/s)
        '''
        return self.cp.PropsSI('A', 'T', T, 'D', rho, self.spec)
    
    def T_IflowSonic(self, T0, P0, P):
        from scipy import optimize
        '''
        solves for the temperature of gas after an isentropic flow, assuming that the entrance
        and exit velocities are sonic.
        
        Parameters
        ----------
        T0 - temperature before expansion (K)
        P0 - pressure before expansion (Pa)
        P - pressure after expansion (Pa)
        
        Returns
        -------
        T - temperature after expansion(K)
        '''
        V0 = self.a(T0, self.rho(T0, P0))
        def errH(T):
            V = self.a(T, self.rho(T, P))
            h = self.cp.PropsSI('H', 'T', T0, 'P', P0, self.spec) + V0**2/2. - V**2/2.
            return h - self.cp.PropsSI('H', 'T', T, 'P', P, self.spec)
        T = optimize.newton(errH, T0)
        return T
    
    def T_AdFlow(self, T0, P0, Ma0, P, Ma):
        from scipy import optimize
        '''
        solves for the temperature of gas after an isentropic flow, assuming that the entrance
        and exit velocities are sonic.
        
        Parameters
        ----------
        T0 - temperature before expansion (K)
        P0 - pressure before expansion (Pa)
        P - pressure after expansion (Pa)
        
        Returns
        -------
        T - temperature after expansion(K)
        '''
        V0 = Ma0*self.a(T0, self.rho(T0, P0))
        def errH(T):
            V = Ma*self.a(T, self.rho(T, P))
            h = self.cp.PropsSI('H', 'T', T0, 'P', P0, self.spec) + V0**2/2. - V**2/2.
            return h - self.cp.PropsSI('H', 'T', T, 'P', P, self.spec)
        T = optimize.newton(errH, T0)
        return T

In [62]:
a=EOS2()

In [63]:
a.cp

<module 'CoolProp.CoolProp' from 'E:\\anaconda\\lib\\site-packages\\CoolProp\\CoolProp.cp37-win_amd64.pyd'>